# RAG Query Evaluation & LLM Training Setup

This notebook demonstrates:
1. How to query the RAG stack programmatically
2. How to evaluate responses
3. How to prepare for LLM fine-tuning
4. How different interfaces share the same infrastructure

## Important: Run Cells in Order
This notebook relies on the Docker-based Jupyter server. Before running:
1. Make sure your stack is up (`make up` or `make smoke`)
2. Run all cells in sequence (Shift + Enter)
3. Watch for any connection errors in the environment check

## Setup: Installing Client Libraries

While our Weaviate server is running in a container, we need its Python client library in our Jupyter environment to talk to it:
- Weaviate Container = Database server (already running)
- `weaviate-client` = Python package to communicate with the server
- Communication happens over HTTP/REST

Let's install the client library:

In [ ]:
# Install the Weaviate client library
import sys
!{sys.executable} -m pip install weaviate-client

print("\nVerifying installation:")
!{sys.executable} -m pip show weaviate-client

## Package Installation
First, we need to install the required Python packages in our Jupyter environment:

## ⚠️ Kernel Selection Is Critical

This notebook MUST use the containerized Jupyter kernel:
1. Select "Existing Jupyter Server" when choosing kernel
2. Connect to `http://localhost:8889` with token `dev`
3. Do NOT use local Mac kernels - they won't work!

Why? The containerized kernel:
- Has access to Weaviate and Ollama services
- Uses correct Docker networking
- Has all dependencies installed
- Uses correct paths (`/home/jovyan/...`)

If you see connection errors or missing files, check your kernel first!

In [1]:
# First, check our environment
import os
import sys
from pathlib import Path

print("Step 1: Checking mounted volumes...")
print("Working directory:", os.getcwd())
print("\nMounted volumes:")
print("- notebooks:", os.path.exists('/home/jovyan/work'))
print("- scripts:", os.path.exists('/home/jovyan/scripts'))
print("- data:", os.path.exists('/home/jovyan/data'))

print("\nStep 2: Setting up connections...")
# Add scripts to path
sys.path.append('../scripts')

# Import required libraries
import weaviate
from rag_query import query_rag

# Initialize Weaviate client
print("Connecting to Weaviate...")
client = weaviate.Client('http://localhost:8080')

# Test connection
try:
    client.is_ready()
    print("Weaviate connection: OK")
except Exception as e:
    print("Weaviate connection failed:", str(e))
    print("\nTroubleshooting tips:")
    print("1. Make sure you ran 'make up' or 'make smoke'")
    print("2. Check if Weaviate is healthy: 'docker compose ps weaviate'")
    print("3. If needed, restart the stack: 'make down && make up'")

Step 1: Checking mounted volumes...
Working directory: /home/jovyan

Mounted volumes:
- notebooks: True
- scripts: True
- data: True

Step 2: Setting up connections...


ModuleNotFoundError: No module named 'weaviate'

In [ ]:
# Verify we're using the correct (containerized) kernel
import socket
import os

def check_environment():
    is_container = os.path.exists('/.dockerenv')
    hostname = socket.gethostname()
    
    if is_container and 'jupyter' in hostname.lower():
        print("✅ Using correct containerized kernel")
        return True
    else:
        print("❌ WARNING: Not using containerized kernel!")
        print(f"Hostname: {hostname}")
        print("This notebook requires the Docker-based Jupyter kernel.")
        print("Please select 'Existing Jupyter Server' and connect to http://localhost:8889")
        return False

in_container = check_environment()

## 1. Query the Existing Knowledge Base

First, let's verify we can access the same data as our scripts:

In [ ]:
# Test query using our shared infrastructure
response = query_rag("What is this project about?")
print("Response:", response)

## 2. Evaluate Response Quality

We can create evaluation metrics to assess our RAG system:

In [ ]:
def evaluate_response(question, response, context=None):
    """Basic evaluation of response quality"""
    metrics = {
        'length': len(response),
        'contains_context': context in response if context else None,
        # Add more metrics as needed
    }
    return metrics

# Example evaluation
test_q = "What components make up this stack?"
test_response = query_rag(test_q)
eval_results = evaluate_response(test_q, test_response)
print("Evaluation:", eval_results)

## 3. Prepare for LLM Training

We can use our RAG results to create training data for fine-tuning:

In [ ]:
def create_training_example(question, response, context):
    """Format a Q&A pair for LLM training"""
    return {
        'instruction': f"Given context: {context}\nQuestion: {question}",
        'input': '',  # Optional additional input
        'output': response
    }

# Collect some examples
training_data = []
test_questions = [
    "What is RAG?",
    "How do I start the stack?",
    "What components are included?"
]

for q in test_questions:
    response = query_rag(q)
    # In practice, you'd also get the relevant context
    example = create_training_example(q, response, "<context>")
    training_data.append(example)

print(f"Created {len(training_data)} training examples")

## 4. Interface Coordination

Demonstrate how changes persist across interfaces:

In [ ]:
# Add a new document - will be available to all interfaces
new_text = """This is a test document about interface coordination.
It demonstrates how changes made in notebooks persist to other interfaces."""

with open('../data/raw/test_coord.txt', 'w') as f:
    f.write(new_text)

# In practice, you'd run the ingestion script here
print("Added new document - accessible from all interfaces")